This notebook aims to contain all functions for indicators.

In [1]:
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import gumbel_r
from scipy.stats import gumbel_l
import os
import os.path
import math

# Precipitation

In [2]:
# import data

In [3]:
out_path = r'\\COWI.net\projects\A245000\A248363\CRVA\Datasets\NEX-GDDP-CMIP6'
csv_path = os.path.join(out_path,'csv_file','pr_day_2021-2060','All_projects_moz_precipitation_2021-2060_without_month_year.csv')
precipitation_2021_2060 = pd.read_csv(csv_path,header=[0,1],index_col=[0,1,2,3,4])*86400 # units kg.m^(-2).s^(-1)
# convert precipitation data from kg.m^(-2).s^(-1) to mm/day :  1 kg/m2/s = 86400 mm/day
# source: https://www.researchgate.net/post/How-do-I-convert-ERA-Interim-precipitation-estimates-from-kg-m2-s-to-mm-day
list_time_2021_2060=pd.date_range('01-01-2021','31-12-2060', freq='D').strftime('%d-%m-%Y').values.tolist()

In [4]:
precipitation_2021_2060_copy = precipitation_2021_2060.copy(deep=True)

In [5]:
df_year=precipitation_2021_2060_copy.reset_index()

In [6]:
df_year[df_year['Date'].str.contains('2021')]

Name project Experiment       Model        Date Latitude  \
                                                                             
0        PT_Revubue_2_Rev_2_01     ssp245  ACCESS-CM2  01-01-2021  -16.125   
1        PT_Revubue_2_Rev_2_01     ssp245  ACCESS-CM2  02-01-2021  -16.125   
2        PT_Revubue_2_Rev_2_01     ssp245  ACCESS-CM2  03-01-2021  -16.125   
3        PT_Revubue_2_Rev_2_01     ssp245  ACCESS-CM2  04-01-2021  -16.125   
4        PT_Revubue_2_Rev_2_01     ssp245  ACCESS-CM2  05-01-2021  -16.125   
...                        ...        ...         ...         ...      ...   
6121950       PT__Dumping_Site     ssp370     TaiESM1  27-12-2021  -13.125   
6121951       PT__Dumping_Site     ssp370     TaiESM1  28-12-2021  -13.125   
6121952       PT__Dumping_Site     ssp370     TaiESM1  29-12-2021  -13.125   
6121953       PT__Dumping_Site     ssp370     TaiESM1  30-12-2021  -13.125   
6121954       PT__Dumping_Site     ssp370     TaiESM1  31-12-2021  -13.125   

         Longitude                           
            33.625 39.875 36.875     40.375  
0         5.057179    NaN    NaN        NaN  
1        17.624178    NaN    NaN        NaN  
2         7.632648    NaN    NaN        NaN  
3         4.736492    NaN    NaN        NaN  
4         4.981353    NaN    NaN        NaN  
...            ...    ...    ...        ...  
6121950        NaN    NaN    NaN   2.239485  
6121951        NaN    NaN    NaN  11.965019  
6121952        NaN    NaN    NaN   8.953759  
6121953        NaN    NaN    NaN   1.699712  
6121954        NaN    NaN    NaN   0.000000  

[153300 rows x 9 columns]

In [ ]:
# return period for each project, model, scenario

In [ ]:
# define function that with adjustement gumbel r

In [ ]:
test=precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0],precipitation_2021_2060_copy.index.levels[1][0],precipitation_2021_2060_copy.index.levels[2][0])]
test=test.droplevel(level=1) # drop latitude index
test=test[[('Longitude','36.875')]]
test.columns = test.columns.droplevel(0) # drop first level of column name
test=test.rename(columns={test.columns[0]:'Precipitation mm/day'})
test = test.filter(like = str(2021), axis=0) # select only data for one year
test#['Precipitation mm'].values

In [ ]:
# test indicator return period of current 50 and 100 year event

In [ ]:
# function value for return period

In [ ]:
def value_return_period(Z,T):
    (loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
    # gumbel_r is chosen because
    p_non_exceedance = 1 - (1/T)
    try:
        threshold_coresponding = round(gumbel_r.ppf(p_non_exceedance,loc,scale))
    except OverflowError: # the result is not finite
        if math.isinf(gumbel_r.ppf(p_non_exceedance,loc,scale)) and gumbel_r.ppf(p_non_exceedance,loc,scale)<0:
            # the result is -inf
            threshold_coresponding = 0 # the value of wero is imposed
    return threshold_coresponding
    # ppf: Percent point function
    #print('Threshold '+str(threshold_coresponding)+' mm/day will be exceeded at least once in '+str(n)+' year, with a probability of '+str(round(p_exceedance*100))+ ' %')
    #print('This threshold corresponds to a return period of '+str(round(return_period))+ ' year event over a '+str(n)+' year period')

In [ ]:
year5_value=value_return_period(test['Precipitation mm/day'].values,5)

In [ ]:
year5_value

In [ ]:
Z = test['Precipitation mm/day'].values
Z.sort()
(loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
# xaxis is precipitation and yaxis is densiy of probability
myHist = plt.hist(Z,density=True) # If ``True``, draw and return a probability density: each bin 
# will display the bin's raw count divided by the total number of counts *and the bin width*
h = plt.plot(Z,gumbel_r.pdf(Z,loc,scale))
plt.xlabel('Precipitation value mm/day')
plt.ylabel('Density of probability' )
plt.title('Histogram and probability density function of precipitation values\nfor year 2021 for one project ,\n one scenario and one model',fontdict={'fontsize': 10})
plt.legend(['Probability density function','Histogramm'])
title_png = 'test_density.png'
path_figure = os.path.join(out_path,'figures')
if not os.path.isdir(path_figure):
    os.makedirs(path_figure)
#plt.savefig(os.path.join(path_figure,title_png),format ='png')
plt.show()

In [ ]:
# accross models and scenarios

In [ ]:
test=precipitation_2021_2060_copy.loc[(precipitation_2021_2060_copy.index.levels[0][0])]
test=test[[('Longitude','36.875')]]
test=test.droplevel(level=3) # drop latitude index
test.columns = test.columns.droplevel(0) # drop first level of column name
test=test.rename(columns={test.columns[0]:'Precipitation mm/day'})
test=test.swaplevel(0,2,axis=0)
test = test.filter(like = str(2021), axis=0) # select only data for one year
#test#['Precipitation mm'].values
test.drop(labels='NESM3',level=1,inplace=True)

In [ ]:
Z = test['Precipitation mm/day'].values
Z.sort()
(loc,scale)=stats.gumbel_r.fit(Z) # return the function necessary to establish the continous function
# xaxis is precipitation and yaxis is densiy of probability
myHist = plt.hist(Z,density=True) # If ``True``, draw and return a probability density: each bin 
# will display the bin's raw count divided by the total number of counts *and the bin width*
h = plt.plot(Z,gumbel_r.pdf(Z,loc,scale))
plt.xlabel('Precipitation value mm/day')
plt.ylabel('Density of probability' )
plt.title('Histogram and probability density function of precipitation values\nfor year 2021 for one project ,\n with all scenarios and models',fontdict={'fontsize': 10})
plt.legend(['Probability density function','Histogramm'])
title_png = 'test_density.png'
path_figure = os.path.join(out_path,'figures')
if not os.path.isdir(path_figure):
    os.makedirs(path_figure)
#plt.savefig(os.path.join(path_figure,title_png),format ='png')
plt.show()

In [ ]:
precipitation_2021_2060_copy.index.levels[0]

In [ ]:
# questions Temps retour :
#      tjs avec maximum ? oui
#      besoin de caler mieux distribution ? package pour le faire automatiquement ?

In [ ]:
years=[precipitation_2021_2060_copy.index.levels[3].values[i][6:10] for i in np.arange(0,len(precipitation_2021_2060_copy.index.levels[3]))]

In [ ]:
years

In [ ]:
len(years)

In [ ]:
len(precipitation_2021_2060_copy.index.levels[3])

In [ ]:
precipitation_2021_2060_copy.index.levels[3]

In [ ]:
precipitation_2021_2060_copy.groupby(like='2021')

In [ ]:
# indicator average per year